# Using the model on images

In order to get an overview of the project, we recommend reading the [README.md](README.md) file.

First we need some imports:
* we use ultralytics yolov8 
* we have already trained a models which we use (the pt files)
* we use Images from IPython
* os is used to get lists of what is contained inside folders

In [ ]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image
import os

The different models we have are printed below. The `v12` models are trained to both detect buildings and classify the color. The `building_superclass.pt` model is trained to detect all buildings in general, and can be used for further development if you want to use another, more accurate approach to classify the color.

In [ ]:
rel_model_path = "data/models"
options = os.listdir(rel_model_path)
print(f"Options: {options}")

Loading in the model weights in one line

In [ ]:
model = YOLO(f"{rel_model_path}/v12_larger.pt")

Showing an easy example of predicting an example image

In [ ]:
rel_building_path = "data/buildings"
img_path = f"{rel_building_path}/examples/image.png"
ClassifiedImage = model(img_path, save=True, project="./outputs", name="saved_image")

# In order to not save:
# ClassifiedImage = model(img_path)

In [ ]:
# Show the image
Image(filename=f"outputs/saved_image/image.png")

Example of how one can collect and print boxes from the result 

In [ ]:
for result in ClassifiedImage:
    # Dimensions of the image
    print(result.orig_shape)
    imgWidth = result.orig_shape[0]
    imgHeight = result.orig_shape[1]
    for box in result.boxes:
        x, y, w, h = box.xywh[0]
        predictedClass = model.names[int(box.cls)]
        confidence = round(float(box.conf), 3)
        print(f"Class: {predictedClass}, confidence: {confidence}")
        print(f"Left {x}, Top {y}, Width {w}, Height {h}")
        
        # Want a measure of how far away the box is from the center of the image
        x_offset = imgWidth/2 - (x + w/2)
        y_offset = imgHeight/2 - (y + h/2)
        print(f"Offset from center: {abs(x_offset)+ abs(y_offset)}")
        print()

In order to get a sense of how good the boxes are, we pick the one with the least offset from the center, using euclidean distance. We also print the confidence of the box. This is the confidence of the model that the box contains a building.

In [ ]:
def gradeResults(result):
    predictions = []
    
    imgWidth = result[0].orig_shape[0]
    imgHeight = result[0].orig_shape[1]
    
    for box in result[0].boxes:
        predictedClass = model.names[int(box.cls)]
        confidence = round(float(box.conf), 3)
        
        x, y, w, h = box.xywh[0]
        x_offset = imgWidth/2 - (x)
        y_offset = imgHeight/2 - (y)
        offset =  x_offset**2 + y_offset**2
        predictions.append((predictedClass, confidence, offset))
    # Outputs list of guesses, confidence, and offset from center.
    return predictions

In [ ]:
def chooseTheMiddleOne(predictions):
    if len(predictions) == 0:
        return "empty"
    # Sorts by offset from center
    predictions.sort(key=lambda x: x[2])
    
    minAcceptableOffset = 200
    if predictions[0][2] > minAcceptableOffset**2:
        return "bad"
    
    MiddleBoxClass = predictions[0][0]
    return MiddleBoxClass, predictions[0][1]

In [ ]:
example_predictions = gradeResults(ClassifiedImage)
predicted_color = chooseTheMiddleOne(example_predictions)
print(f"Predicted Color of {img_path} was {predicted_color}")

Example using the model on the four cardinal directions

In [ ]:
predictions = []
for i in range(1, 5):
    PathToImage = f"{rel_building_path}/four_directions_1/{str(i)}.png"
    result = model(PathToImage)
    pred = gradeResults(result)
    predictedClass = chooseTheMiddleOne(pred)
    predictions.append(predictedClass)
    
# Make the predicted boxes visible in order to check the predictions
folder_path = f"{rel_building_path}/four_directions_1"
model(folder_path, save=True, project="./outputs", name="four_dir_1")

print(f"The resulting predictions: {predictions}")

display(Image(filename=f"outputs/four_dir_1/1.png", height=100))
display(Image(filename=f"outputs/four_dir_1/2.png", height=100))
display(Image(filename=f"outputs/four_dir_1/3.png", height=100))
display(Image(filename=f"outputs/four_dir_1/4.png", height=100))

Same with different test set

In [ ]:
# predictions = []
# for i in range(1, 5):
#     PathToImage = f"{rel_building_path}/four_directions_2/{str(i)}.png"
#     result = model(PathToImage)
#     pred = gradeResults(result)
#     predictedClass = chooseTheMiddleOne(pred)
#     predictions.append(predictedClass)
    
# # Make the predicted boxes visible in order to check the predictions
# folder_path = f"{rel_building_path}/four_directions_2"
# model(folder_path, save=True, project="./outputs", name="four_dir_2")

# print(f"The resulting predictions: {predictions}")

# display(Image(filename=f"outputs/four_dir_2/1.png", height=100))
# display(Image(filename=f"outputs/four_dir_2/2.png", height=100))
# display(Image(filename=f"outputs/four_dir_2/3.png", height=100))
# display(Image(filename=f"outputs/four_dir_2/4.png", height=100))

Testing on a Drone video. This will take very long time to run, so we recommend to run it on Google Colab.


In [ ]:
# model = YOLO("data/models/v12_smaller.pt")
# output = model("data/video/demo_drone.mp4", save=True, project="./outputs", name="drone_v12_smaller")

# Model for finding the buildings, without caring about the class
# model = YOLO("data/models/building_detection.pt")
# output = model("data/video/demo_drone.mp4", save=True, project="./outputs", name="drone_building_detection")